In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import sys
from IPython.display import Image
from shapely.geometry import Point, Polygon
from math import factorial
import datetime
import time
import scipy

from statsmodels.sandbox.regression.predstd import wls_prediction_std
from sklearn.linear_model import LinearRegression
from patsy import cr

from pprint import pprint
import matplotlib.pyplot as plt
import seaborn as sb

In [2]:
import sys
# search path for modules
# look @ https://stackoverflow.com/questions/67631/how-to-import-a-module-given-the-full-path
sys.path.append('/Users/hn/Documents/00_GitHub/Ag/remote_sensing/python/')
import remote_sensing_core as remote_core

# Plan for rough estimate:
   1. perennials, and grasses and non irrigated (Why google slide doess not say annuals)
    

# Function definitions

# Directories

In [3]:
data_dir = "/Users/hn/Documents/01_research_data/Ag_check_point/remote_sensing/01_NDVI_TS/Grant/"

# Data Reading

In [4]:
file_names = ["Grant_2018_TS.csv"]
file_N = file_names[0]

In [5]:
a_df = pd.read_csv(data_dir + file_N)
a_df = remote_core.initial_clean(a_df)
a_df.head(2)

,Acres,B2,B3,B4,B8,CovrCrp,CropGrp,CropTyp,DataSrc,ExctAcr,...,NDVI,Notes,RtCrpTy,Shap_Ar,Shp_Lng,TRS,county,doy,year,geo
0,30,0.743039,0.708845,0.805611,0.728629,NaN,Cereal Grain,Wheat,NASS,30.356486,...,-0.051402,Winter Wheat,Wheat Fallow,122848.340543,2265.483255,T23R26E2,Grant,2.0,2018.0,"{""type"":""Polygon"",""coordinates"":[[[-119.527755..."
1,30,0.600203,0.556632,0.638710,0.617168,NaN,Cereal Grain,Wheat,NASS,30.356486,...,-0.015851,Winter Wheat,Wheat Fallow,122848.340543,2265.483255,T23R26E2,Grant,29.0,2018.0,"{""type"":""Polygon"",""coordinates"":[[[-119.527755..."


In [6]:
# Count distict values, use nunique:
pprint (a_df['geo'].nunique())

# Count only non-null values, use count:
print (a_df['geo'].count())

# Count total values including null values, use size attribute:
print (a_df['geo'].size)

7
378
378


### List of unique polygons

In [7]:
polygon_list = a_df['geo'].unique()

# Iterate through polygons and count peaks

In [20]:
output_columns = ['Acres', 'CovrCrp', 'CropGrp', 'CropTyp',
                  'DataSrc', 'ExctAcr', 'IntlSrD', 'Irrigtn', 'LstSrvD', 'Notes',
                  'RtCrpTy', 'Shap_Ar', 'Shp_Lng', 'TRS', 'county', 'year', 'geo',
                  'peak_Doy', 'peak_value']

all_polygons_and_their_peaks = pd.DataFrame(data=None, 
                                            columns=output_columns)

for a_poly in polygon_list:
    a_field = a_df[a_df['geo']==a_poly]
    
    ### 
    ###  There is a chance that a polygon is repeated twice?
    ###
    
    X = a_field['doy']
    y = a_field['NDVI']
    freedom_df = 7
    #############################################
    ###
    ###             Smoothen
    ###
    #############################################
    
    # Generate spline basis with "freedom_df" degrees of freedom
    x_basis = cr(X, df=freedom_df, constraints='center')

    # Fit model to the data
    model = LinearRegression().fit(x_basis, y)

    # Get estimates
    y_hat = model.predict(x_basis)
    
    #############################################
    ###
    ###             find peaks
    ###
    #############################################
    # peaks_LWLS_1 = peakdetect(LWLS_1[:, 1], lookahead = 10, delta=0)
    # max_peaks = peaks_LWLS_1[0]
    # peaks_LWLS_1 = form_xs_ys_from_peakdetect(max_peak_list = max_peaks, doy_vect=X)

    peaks_spline = remote_core.peakdetect(y_hat, lookahead = 10, delta=0)
    max_peaks =  peaks_spline[0]
    peaks_spline = remote_core.form_xs_ys_from_peakdetect(max_peak_list = max_peaks, doy_vect=X)
    
    DoYs_series = pd.Series(int(peaks_spline[0]))
    peaks_series = pd.Series(peaks_spline[1])
    
    peak_df = pd.DataFrame({ 
                       'peak_Doy': pd.Series(int(peaks_spline[0])), 
                       'peak_value': pd.Series(peaks_spline[1]) 
                      }) 

    
    WSDA_df = remote_core.keep_WSDA_columns(a_field)
    WSDA_df = WSDA_df.drop_duplicates()
    
    WSDA_df = pd.concat([WSDA_df]*peak_df.shape[0]).reset_index()
    # WSDA_df = pd.concat([WSDA_df, peak_df], axis=1, ignore_index=True)
    WSDA_df = WSDA_df.join(peak_df)
    all_polygons_and_their_peaks = all_polygons_and_their_peaks.append(WSDA_df, sort=False)
    
    """
    # first I decided to add all DoY and peaks in one row to avoid
    # multiple rows per (field, year)
    # However, in this way, each pair of (field, year)
    # can have different column sizes.
    # So, we cannot have one dataframe to include everything in it.
    # so, we will have to do dictionary to save out puts.
    # Lets just do replicates... easier to handle perhaps down the road.
    #
    DoY_colNames = [i + j for i, j in zip(\
                                          ["DoY_"]*(len(DoYs_series)+1), \
                                          [str(i) for i in range(1, len(DoYs_series)+1)] )] 
   
    peak_colNames = [i + j for i, j in zip(\
                                           ["peak_"]*(len(peaks_series)+1), \
                                           [str(i) for i in range(1, len(peaks_series)+1)] )]
    
    WSDA_df[DoY_colNames] = pd.DataFrame([DoYs_series], index=WSDA_df.index)
    WSDA_df[peak_colNames] = pd.DataFrame([peaks_series], index=WSDA_df.index)
    """

In [21]:
all_polygons_and_their_peaks

,Acres,CovrCrp,CropGrp,CropTyp,DataSrc,ExctAcr,IntlSrD,Irrigtn,LstSrvD,Notes,RtCrpTy,Shap_Ar,Shp_Lng,TRS,county,year,geo,peak_Doy,peak_value,index
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,80,NaN,Other,CRP/Conservation,NASS,80.408982,2005/01/03 00:00:00,None,2018/12/31 00:00:00,NaN,NaN,325403.603491,2466.296163,T23R26E2,Grant,2018.0,"{""type"":""GeometryCollection"",""geometries"":[{""t...",152,0.247690,110.0
0,3,NaN,Cereal Grain,Wheat Fallow,NASS,3.424051,2005/01/03 00:00:00,None,2018/12/31 00:00:00,NaN,Wheat,13856.641211,773.171544,T27R29E31,Grant,2018.0,"{""type"":""Polygon"",""coordinates"":[[[-119.233717...",152,0.306236,165.0
0,2,NaN,Cereal Grain,Wheat Fallow,NASS,2.093428,2005/01/03 00:00:00,None,2018/12/31 00:00:00,NaN,Wheat,8471.804047,788.041888,T27R29E31,Grant,2018.0,"{""type"":""Polygon"",""coordinates"":[[[-119.233980...",114,0.153380,222.0
0,2,NaN,Herb,Marijuana,WSDA,1.610041,2018/11/09 00:00:00,Drip,2018/11/09 00:00:00,NaN,NaN,6515.606577,325.417942,T22R26E1,Grant,2018.0,"{""type"":""Polygon"",""coordinates"":[[[-119.497041...",109,0.270041,279.0


In [ ]:
WSDA_df = pd.concat([WSDA_df]*peak_df.shape[0]).reset_index(drop=True)
WSDA_df = WSDA_df.join(peak_df)

In [ ]:
print(all_polygons_and_their_peaks.shape)
print(WSDA_df.shape)

In [ ]:
all_polygons_and_their_peaks

In [ ]:
WSDA_df

In [14]:
a_df.shape
len(a_df)

378

,Acres,CovrCrp,CropGrp,CropTyp,DataSrc,ExctAcr,IntlSrD,Irrigtn,LstSrvD,Notes,RtCrpTy,Shap_Ar,Shp_Lng,TRS,county,year,geo,peak_Doy,peak_value
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1129,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1130,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1131,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1132,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
X = a_field['doy']
y = a_field['NDVI']
freedom_df = 7

# Generate spline basis with "freedom_df" degrees of freedom
x_basis = cr(X, df=freedom_df, constraints='center')

# Fit model to the data
model = LinearRegression().fit(x_basis, y)

# Get estimates
y_hat = model.predict(x_basis)

In [ ]:
# find peaks
# peaks_LWLS_1 = peakdetect(LWLS_1[:, 1], lookahead = 10, delta=0)
# max_peaks = peaks_LWLS_1[0]
# peaks_LWLS_1 = form_xs_ys_from_peakdetect(max_peak_list = max_peaks, doy_vect=X)

peaks_spline = peakdetect(y_hat, lookahead = 10, delta=0)
max_peaks = peaks_spline[0]
peaks_spline = form_xs_ys_from_peakdetect(max_peak_list = max_peaks, doy_vect=X)

In [ ]:
plt.scatter(X, y, s=7);
plt.scatter(peaks_spline[0], peaks_spline[1], s=120, c='c', marker='*');
plt.plot(X, y_hat, 'r');
plt.title(f'Natural cubic spline with {freedom_df} degrees of freedom \n and peak point in red');

In [ ]:
peaks_spline

In [ ]:
WSDA_df = convert_TS_to_a_row(a_field)
WSDA_df

In [ ]:
WSDA_df = WSDA_df.reset_index(drop=True)
WSDA_df

In [ ]:
# DoYs_df = pd.Series(int(peaks_spline[0]), name='peak_Doy').to_frame()
# peaks_df = pd.Series(peaks_spline[1], name="peak_value").to_frame()

peak_df = pd.DataFrame({ 
                       'peak_Doy': pd.Series(int(peaks_spline[0])), 
                       'peak_value': pd.Series(peaks_spline[1]) 
                      }) 

peak_df = pd.DataFrame({ 
                       'peak_Doy': np.array([147, 148, 2]), 
                       'peak_value': np.array([10.2, 14.8, 33.4])
                      }) 

peak_df

In [ ]:
# pd.merge(WSDA_df, peak_df)
# pd.merge(left=WSDA_df, right=peak_df, how='outer', on='index')
pd.concat([AAA, peak_df], axis=1, ignore_index=True)

In [ ]:
AAA = pd.concat([WSDA_df]*peak_df.shape[0]).reset_index()
AAA

In [ ]:
peak_df

In [ ]:
AAA.join(peak_df)

In [ ]:
pd.concat([WSDA_df]*peak_df.shape[0]).reset_index()

In [ ]:
peak_df.shape[0]

In [ ]:
peak_df